In [1]:
import pyspark
from pyspark.sql import SparkSession

pyspark.__version__

'3.2.1'

## Finally some progress. Thrift server exposes a jdbc/odbc connection, so you can load/save to it using your local spark context. You can not use it to directly connect to the metastore.

- Now we get a schema error instead
- Should we switch to using a Postgres metadata server and directly connect?

In [2]:
spark = SparkSession.builder\
          .appName("entr_notebook")\
          .config("spark.sql.warehouse.dir", "/home/jovyan/warehouse")\
          .config("spark.hadoop.javax.jdo.option.ConnectionURL", "jdbc:derby:;databaseName=/home/jovyan/warehouse/metastore_db;create=true")\
          .enableHiveSupport()\
          .getOrCreate()

spark.sql("use entr_warehouse")

DataFrame[]

In [3]:
spark.sql("show tables").toPandas()

,namespace,tableName,isTemporary
0,entr_warehouse,dim_asset_reanalysis_dataset,False
1,entr_warehouse,dim_asset_wind_plant,False
2,entr_warehouse,dim_asset_wind_turbine,False
3,entr_warehouse,dim_entr_tag_list,False
4,entr_warehouse,fct_entr_plant_data,False
5,entr_warehouse,fct_entr_reanalysis_data,False
6,entr_warehouse,fct_entr_wtg_scada,False
7,entr_warehouse,int_entr_era5_sample__cast,False
8,entr_warehouse,int_entr_merra2_sample__cast,False
9,entr_warehouse,int_entr_plant_sample__cast,False


## Bad attempts to  connect to the thrift server as a metastore.

In [ ]:
 val ses = SparkSession
          .builder()
          .appName("interfacing spark sql to hive metastore without configuration file")
          .config("hive.metastore.uris", "thrift://localhost:9083")
          .enableHiveSupport()
          .getOrCreate()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
          appName("entr_notebook").\
          config("hive.metastore.uris", "thrift://localhost:9083").\
          enableHiveSupport().\
          getOrCreate()

In [5]:
spark.sql("show databases;")

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

## Try directly reading from metastore_db derby


In [11]:
spark = SparkSession.builder.\
          appName("entr_notebook").\
          config("spark.sql.warehouse.dir", "/home/jovyan/warehouse").\
          config("spark.hadoop.javax.jdo.option.ConnectionURL", "jdbc:derby:;databaseName=/home/jovyan/warehouse/metastore_db").\
          enableHiveSupport().\
          getOrCreate()
spark.sql("show databases")



AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: org.apache.thrift.transport.TTransportException

In [3]:
import findspark
findspark.init()
import pyspark
import random
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *
import time
config = pyspark.SparkConf().setAll([(
    "spark.executor.memory", "64g"), (
    "spark.executor.cores", "8"), (
    "spark.cores.max", "8"), (
    "spark.driver.memory", "64g"), (
    "spark.sql.shuffle.partitions", "50"),
    ("spark.driver.maxResultSize","5g"),
    ("spark.sql.execution.arrow.enabled", "true")])
#    ("spark.driver.extraClassPath”,“/home/jovyan/host/gcs-connector-latest-hadoop2.jar:/home/jovyan/spark-bigquery-with-dependencies_2.12-0.21.1.jar”)])
spark = SparkSession.builder.appName("PysparkExample").config(conf=config).getOrCreate()
sql = SQLContext(spark)
sc = spark.sparkContext
#spark._jsc.hadoopConfiguration().set(“google.cloud.auth.service.account.json.keyfile”,“/home/jovyan/entrhub-entrguest-abf6082b3146.json”)
#spark.sparkContext._jsc.hadoopConfiguration().set(“fs.gs.impl”, “com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem”)
#spark.sparkContext._jsc.hadoopConfiguration().set(“fs.AbstractFileSystem.gs.impl”, “com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS”)
#spark.sparkContext._jsc.hadoopConfiguration().set(“google.cloud.auth.service.account.enable”, “true”)
#spark.sql(“set spark.sql.legacy.timeParserPolicy=LEGACY”)
#bucket = “entrhub”
#spark.conf.set(‘temporaryGcsBucket’, bucket)

In [3]:
spark = SparkSession.builder \
    .master("local")\
    .appName("hive_pyspark")\
    .config("hive.metastore.uris", "thrift://localhost:10000")\
    .enableHiveSupport().getOrCreate()

In [5]:
spark = SparkSession.builder \
.master("local")\
.appName("hive_pyspark")\
.config("hive.metastore.uris", "jdbc:hive2://localhost:10000")\
.getOrCreate()

AttributeError: 'Builder' object has no attribute 'format'

In [2]:
df=spark.sql("show databases")
df.show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [4]:
df = spark.read.option("url", "jdbc:hive2://localhost:10000").option("dbtable", "people").format("jdbc").load()

Py4JJavaError: An error occurred while calling o165.load.
: java.sql.SQLException: org.apache.hive.service.cli.HiveSQLException: Error running query: org.apache.spark.sql.AnalysisException: Table or view not found: people; line 1 pos 14;
'Project [*]
+- 'Filter (1 = 0)
   +- 'UnresolvedRelation [people], [], false

	at org.apache.spark.sql.hive.thriftserver.HiveThriftServerErrors$.runningQueryError(HiveThriftServerErrors.scala:43)
	at org.apache.spark.sql.hive.thriftserver.SparkExecuteStatementOperation.org$apache$spark$sql$hive$thriftserver$SparkExecuteStatementOperation$$execute(SparkExecuteStatementOperation.scala:325)
	at org.apache.spark.sql.hive.thriftserver.SparkExecuteStatementOperation$$anon$2$$anon$3.$anonfun$run$2(SparkExecuteStatementOperation.scala:230)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.sql.hive.thriftserver.SparkOperation.withLocalProperties(SparkOperation.scala:79)
	at org.apache.spark.sql.hive.thriftserver.Sp